In [2]:
import pandas as pd
import numpy as np

In [3]:
data_g2 = pd.read_csv("./Data/Kaohsiung/g2.csv",delimiter=';')
data_g3 = pd.read_csv("./Data/Kaohsiung/g3.csv",delimiter=';')
data_g4 = pd.read_csv("./Data/Kaohsiung/g4.csv",delimiter=';')

data = pd.concat([data_g2, data_g3, data_g4], ignore_index=True)

In [4]:
print(data)

                  date_time            E             N          H  Angle  \
0       2024-01-18 04:30:00  224095.7991  2.551570e+06  1518.4140    0.0   
1       2024-01-18 04:40:00  224095.7980  2.551570e+06  1518.4135  137.0   
2       2024-01-18 10:40:00  224095.7973  2.551570e+06  1518.3954  151.0   
3       2024-01-18 10:50:00  224095.8001  2.551570e+06  1518.3838  277.0   
4       2024-01-18 11:10:00  224095.8011  2.551570e+06  1518.3934  288.0   
...                     ...          ...           ...        ...    ...   
120379  2024-12-30 11:00:00  224204.5244  2.551598e+06  1538.2650  292.0   
120380  2024-12-30 11:10:00  224204.5241  2.551598e+06  1538.2652  297.0   
120381  2024-12-30 11:20:00  224204.5241  2.551598e+06  1538.2660  292.0   
120382  2024-12-30 11:30:00  224204.5235  2.551598e+06  1538.2646  282.0   
120383  2024-12-30 11:40:00  224204.5237  2.551598e+06  1538.2650  282.0   

        Axis  Plate  EMove   NMove   HMove  TotalMove         EDay  \
0        0.0    0

In [5]:
data['date_time'] = pd.to_datetime(data['date_time'])
data['date_time'] = data['date_time'].astype(int) / (10 ** 11)

In [6]:
print(data['date_time'])

0         17055522.0
1         17055528.0
2         17055744.0
3         17055750.0
4         17055762.0
             ...    
120379    17355564.0
120380    17355570.0
120381    17355576.0
120382    17355582.0
120383    17355588.0
Name: date_time, Length: 120384, dtype: float64


In [7]:
lag_features = ['E', 'N', 'H', 'TotalMove']
lag_steps = 3

for feature in lag_features:
    for lag in range(1, lag_steps + 1):
        data[f'{feature}_lag{lag}'] = data[feature].shift(lag)

data = data.dropna()

In [8]:
target = ['E', 'N', 'H', 'TotalMove']
X = data.drop(columns = target)
y = data[target]

In [9]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xgbrModel=xgb.XGBRegressor()
xgbrModel.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [11]:
y_pred = xgbrModel.predict(X_test)

In [12]:
print(y_pred)

[[2.2420528e+05 2.5516008e+06 1.5411913e+03 2.5962112e+01]
 [2.2409580e+05 2.5515700e+06 1.5183749e+03 2.4909477e+02]
 [2.2420527e+05 2.5516008e+06 1.5411730e+03 5.3027222e+01]
 ...
 [2.2417609e+05 2.5515630e+06 1.5258491e+03 8.3621712e+01]
 [2.2409580e+05 2.5515700e+06 1.5183634e+03 2.7006461e+02]
 [2.2417609e+05 2.5515630e+06 1.5258667e+03 6.5820892e+01]]


In [13]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

E_mae = mean_absolute_error(y_test['E'], y_pred[:,0])
E_rmse = np.sqrt(mean_squared_error(y_test['E'], y_pred[:,0]))

N_mae = mean_absolute_error(y_test['N'], y_pred[:,1])
N_rmse = np.sqrt(mean_squared_error(y_test['N'], y_pred[:,1]))

H_mae = mean_absolute_error(y_test['H'], y_pred[:,2])
H_rmse = np.sqrt(mean_squared_error(y_test['H'], y_pred[:,2]))

TotalM_mae = mean_absolute_error(y_test['TotalMove'], y_pred[:,3])
TotalM_rmse = np.sqrt(mean_squared_error(y_test['TotalMove'], y_pred[:,3]))

results = pd.DataFrame({
    "Variable": ["E","N","H","TotalMove"],
    "MAE": [E_mae,N_mae,H_mae,TotalM_mae],
    "RMSE": [E_rmse,N_rmse,H_rmse,TotalM_rmse]
})

print(results)

    Variable       MAE       RMSE
0          E  0.017004   0.083194
1          N  0.252455   0.260614
2          H  0.004676   0.045304
3  TotalMove  4.034556  31.644797
